# Notes on LDA

- Compiled from Resources listed at the bottom
- Tovio Roberts
- 2018, October
- tovioroberts@gmail.com
    - please contact me to suggest edits, as this is meant to be a learning resource

- **topic**: probability distribution over a collection of words, a thematic summary
- **topic model**: formal statistical relationship betw a group of observed and latent (unknown) random variables that specifies a probabilistic procedure to generate the topics

### Generative Model
- **dataset**: collection of _D_ documents
- **document**: a collection of words
    - in LDA, contains multiple topics
    - in LDA, order of words is not not maintained (b.o.w.)
- **vocabulary**: all unique words in the corpus
    - _V_ : number of terms in the corpus
- assume we know _K_ topic distributions in the dataset
    - _K_ multinomials containing _V_ elements each
    - $\beta_1$ : represents the multinomial for the $i$th topic
        - size of $\beta_1$ is $V: \mid \beta_1 \mid = V$

#### How LDA assumes documents are created:
1. Determine number of words in a document
2. Choose a topic mixture for the document over a fixed set of topics (25% Topic 0, 45% Topic 1, 30% Topic 2)
3. Generate the words in the document:
    - a. first pick a topic based on the document's multinomial distribution from step 2
    - b. pick a word based on the topic's multinomial distribution

### LDA Statistical Inference Procedure for learning the topics and topic distributions
- Given a Corpus of documents, LDA learns the topic representation in each document and the word distribution of each topic
    - LDA identifies the topics that are likely to have generated the corpus

#### High-level view of the LDA process
- **Essentially updating assignments of words with evolving model of how documents are created**
1. Randomly assign each word in each document to one of the _k_ topics
2. For each document $d$:
    - Assume all topic assignments except for current one to be correct
    - Calculate 2 proportions
        1. Proportion of words in document $d$ that are currently assigned to topic $t = p(\text{topic } t \mid \text{document }d)$
        2. Proportion of assignments to topic $t$ over all documents that came from this word $w = p(\text{word }w \mid \text{topic }t)$
    - multiply the proportions and assign $w$ a new topic based on that probability.
        - $p(\text{topic } t \mid \text{document }d) \times p(\text{word } w \mid \text{topic }t)$
            - probability that topic $t$ generated word $w$ in a given document
3. Reach steady state and return parameters
    - topic mixtures of each of the documents
    - underlying word distributions for topics


#### Mid-level view of the LDA process
- For each document $d$
    - (a) draw a topic distribution, $\theta_d \sim Dir(\alpha)$, where $Dir(\cdot)$ is a draw from a _uniform_ Dirichlet distribution with scaling parameter $\alpha$
        - this a draw from a _k_ dimensional Dirichlet distribution which returns a _k_ dimensional multinomial $\theta$ where the _k_ values sum to 1
    - (b) for each word in the document:
        - (i) Draw a specific topic $z_{d,n} \sim multi(\theta_d)$ where $multi(\cdot)$ is a multinomial
        - (ii) Draw a word $w_{d,n} \sim \beta_{z_{d,n}}$

#### Prob Density of k-dimensional multinomial $\theta$ in LDA
$$
p(\theta \mid \alpha) = \frac{\Gamma (\sum_{i=1}^k \alpha_i)}{\prod_{i=1}^k \Gamma(\alpha_i)}\prod_{i=1}^k \theta_i^{\alpha_i - 1}
$$
- $\theta$ is normalized ensuring it lies on a $(k-1)$ dimensional simplex

#### LDA Graphical Model
- using plate notation, a way of visually representing the dependencies among the model parameters

![LDA Graphical Model](images\lda_graphical_model.png)

- $M$ : denotes lg rectangle, reps total number of documents in the corpus
- $N$ : smaller rectangle, total number of words in a document
- $\theta_d$ : topic distribution for document $d$ of $M$
- $w$ : a word
    - $w_{d,n}$ : a specific word
    - $w^v$ : the $v$th word in the vocabulary
    - $\mathbf w$ : a document/vector of words, $w = (w_1, w_2, ..., w_N)$
- $\mathbf z$ : vector of topics from which $w$ draws, used to notate each topic, which is assigned to each word, making each document a mixture of these topics
    - $z_{d,n}$ : topic for the $n$th word in document $d$ 
- Dirichlet Priors: $\alpha$ and $\beta$
    - $\alpha$ : parameter of Dirichlet prior on the per-document topic distributions
        - high $\alpha$ indicates that each document is likely to contain a mixture of most of the topics, not 1 or 2 in particular
            - low $\alpha$ indicates that only a couple or few topics will be dominant
        - $\alpha = (\alpha_1, \alpha_2, ..., \alpha_k)$
        - all elems of $\alpha$ will usually be the same and can be rep'd in code by a single number
    - $\beta$ : $k \times V$ word-probability matrix
        - parameter of the Dirichlet prior on the per-topic word distribution
            - high $\beta$ indicates that each topic will contain a mixture of most of the words, low indicates each topic contains a mixture of just a few of the words
        - row : topic
        - column : term
        - $\beta_{ij} = p(w^j = 1 \mid z^i = 1)$

#### Central Inferential Problem : Determine the Posterior Distr of the latent variables given the document
$$
p(\theta, \mathbf z \mid \mathbf w, \alpha, \beta) = \frac{p(\theta, \mathbf z, \mathbf w \mid \alpha, \beta)}{p(\mathbf w \mid \alpha, \beta)}
$$

$$
\Downarrow
$$

$$
\text{decompose numerator into hierarchy by examining graphical model}
$$

$$
\Downarrow
$$

$$
p(\theta, \mathbf z, \mathbf w \mid \alpha, \beta) = p(\mathbf w \mid \mathbf z, \beta)p(\mathbf z \mid \theta)p(\theta \mid \alpha)
$$

$$
\text{where } p(\mathbf w \mid \mathbf z, \beta) \\
\text{ represents the probability of observing a document with } N \\
\text{ words given the topic vector of length } N \text{ that assigns a topic to each word from the } \\ 
k \times V \text{ probability matrix } \beta \\
\text{thus we can decompose this into each individual} \\
\text{word probability and multiply them together:}
$$

$$
p(\mathbf w \mid \mathbf z, \beta) = \prod_{n=1}^N \beta_{z_n, w_n}
$$

<hr>

$$
p(\mathbf z \mid \theta) \text{ is trivial:} \\
p(z_n \mid \theta) = \theta_i, \text{ such that } z_n^i = 1 \\
\text{ as }\theta \text{ is a multinomial}
$$

<hr>

$$
\text{recall: } \\
p(\theta \mid \alpha) = \frac{\Gamma (\sum_{i=1}^k \alpha_i)}{\prod_{i=1}^k \Gamma(\alpha_i)}\prod_{i=1}^k \theta_i^{\alpha_i - 1}
$$

<hr>

$$
\text{using } \theta_{z_n} \text{ to rep component of } \theta \text{ chosen for } z_n : \\
p(\theta, \mathbf z, \mathbf w \mid \alpha, \beta) = \Biggl( \frac{\Gamma ( \sum_{i=1}^k \alpha_i)}{\prod_{i=1}^k \Gamma (\alpha_i)}\prod_{i=1}^k \theta_i^{\alpha_i - 1} \Biggr) \prod_{n=1}^N \beta_{z_n, w_n} \theta_{z_n}
$$

$$
\Downarrow
$$

$$
\text{rephrased using word-indexing supercript }w^v \text{ and words used as exponent}
$$

$$
\Downarrow
$$

$$
p(\theta, \mathbf z, \mathbf w \mid \alpha, \beta) = \Biggl( \frac{\Gamma ( \sum_{i=1}^k \alpha_i)}{\prod_{i=1}^k \Gamma (\alpha_i)}\prod_{i=1}^k \theta_i^{\alpha_i - 1} \Biggr) \prod_{n=1}^N \prod_{i=1}^k \prod_{j=1}^V ( \theta_i \beta_{i,j})^{w_n^j z_n^i}
$$

$$
\Downarrow
$$

$$
\text{marginalize over } \theta \text{ and } z \text{ in order to obtain the denominator (the evidence)}
$$

$$
\Downarrow
$$

$$
p(\mathbf w \mid \alpha, \beta) = \frac{\Gamma ( \sum_{i=1}^k \alpha_i)}{\prod_{i=1}^k \Gamma (\alpha_i)} \int \Biggl( \prod_{i=1}^k \theta_i^{\alpha_i - 1} \Biggr) \Biggl( \prod_{n=1}^N \sum_{i=1}^k \prod_{j=1}^V (\theta_i \beta_{ij})^{w_{n}^j} \Biggr) d \theta
$$
- this final expression cannot be computed, as the inability to separate $\theta$ and $\beta$ prevents computing the log of the function, so use variational inference techniques in practice

#### Variational Inference for LDA
- `variational inference` : use simpler, convex distribution that obtains an adjustable lower bound on the log likelihood of the actual distribution
- `variational parameterss` : describe the family of simpler distributions used to determine a lower bound on the log likelohood, optimized to create the tightest possible lower bound

##### _Drop coupling betw $\theta$ and $\beta$ to make inference possible_

![LDA Graphical Model](images\lda_graphical_model.png)
$$
\Downarrow
$$

$$
\text{drop some edges and nodes from fig 2 to obtain simplified graphical model}
$$

$$
\Downarrow
$$

![Variational LDA Graphical Model](images\variational_lda_graphic.png)

$$
\text{posterior for each document, from the variational distribution} \\
q(\theta, \mathbf z \mid \gamma, \phi) = q(\theta \mid \gamma) \prod_{n=1}^N q(z_n \mid \phi_n)
$$

- **Find an optimal lower bound on log likelihood results using minimization of the Kellback-Leibler (KL) divergence betw the variational distribution and the actual posterior distribution:**


### Using Expectation Maximization Algorithm on Variational Distribution

- **Expectation Maximization**
    - use EM to discover parameters of the prob distribution, automatically discover all parameters for the _K_ sources
    - mixture model used to perform `soft clustering`
        - `hard clustering`: no overlap of clusters
        - `soft clustering`: clusters may overlap
    - each cluster corresponds to a prob distribution
    - Unlike KMeans, does not assign to hard cluster
    - iterates until convergence

- **EM Steps**: E-Step, M-Step
    - Estimate $\beta$ and $\alpha$ assuming we know $\phi$ $\gamma$
        - $\alpha$ : parameter of Dirichlet distribution
        - $\beta$ : $k \times V$ word-probability matrix
    - `E-Step` : detm log likelihood of the complete data, estimate hidden parameters $\alpha$ and $\beta$ by finding the optimal values of the variational parameters $\gamma_d^*$ and $\phi_d^*$ for every document in the corpus
    - `M-Step` : Maximize lower bound on the log likelihood $l(\alpha, \beta) = \sum_{d=1}^M log p(\mathbf w_d \mid \alpha, \beta)$ with respect to $\alpha$ and $\beta$
        - corresponds to finding maximum likelihood estimates with expected sufficient statistics for each document
- **Normalization/Scaling**
    - normalize all $\beta_i$ to sum to one
    - use linear-scaling Newton-Rhapson algorithm to determine optimal $\alpha$, updates carried out in log-space

#### Pseudocode for Variational Expectation-Maximization LDA, [From Reed 2012](http://obphio.us/pdfs/lda_tutorial.pdf)


##### Input:
Number of topics $K$<br>
Corpus with $M$ documents and $N_d$ words in document $d$<br>

##### Output:
Model parameters: $\beta, \theta, z$<br>

##### Setup
initialize $\phi_{ni}^0$ := 1/$k$ for all $i$ in $k$ and $n$ in $N_d$<br>
initialize $\gamma_i$ := $\alpha_i + N/k$ for all $i$ in $k$<br>
initialize $\alpha$ := $50/k$<br>
initialize $\beta_{ij}$ := $0$ for all $i$ in $k$ and $j$ in $V$<br>

##### E-Step: Determine $\phi$ and $\gamma$ and compute expected log likelihood
**for** $d = 1$ **to** $M$<br>
&nbsp;&nbsp;&nbsp;&nbsp;**repeat**<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**for** $n = 1$ **to** $N_d$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **for** $i=1$ **to** $K$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\phi_{dni}^{t+1}$ := $\beta_{iw_n} exp(\Psi(\gamma_{di}^t))$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **endfor**<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; normalize $\phi_{dni}^{t+1}$ to sum to 1<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**endfor**<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\gamma^{t+1}$ := $\alpha + \sum_{n=1}^N \phi_{dn}^{t+1}$<br>
&nbsp;&nbsp;&nbsp;&nbsp;**until** convergence of $\phi_d$ and $\gamma_d$<br>
&nbsp;&nbsp;&nbsp;&nbsp;loglikelihood := loglikelihood + $L(\gamma, \phi; \alpha, \beta)$
**endfor**<br>

##### M-Step: maximize the log likelohood of the variational distribution
**for** $d=1$ **to M**<br>
&nbsp;&nbsp;&nbsp;&nbsp;**for** $i=1$ **to K**<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**for** $j = 1$ **to** $V$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\beta_{ij}$ := $\phi_{dni} w_{dnj}$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**endfor**<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;normalize $\beta_i$ to sum to 1<br>
&nbsp;&nbsp;&nbsp;&nbsp;**endfor**<br>
**endfor**<br>
estimate $\alpha$ via $log(\alpha^{t+1}) = log(\alpha^t) - \frac{\frac{dL}{d\alpha}}{\frac{d^2L}{d\alpha^2} \alpha + \frac{dL}{d\alpha}}$<br>
**if** loglikelihood converged **then**<br>
&nbsp;&nbsp;&nbsp;&nbsp;return parameters<br>
**else**<br>
&nbsp;&nbsp;&nbsp;&nbsp;go back to E-step<br>
**endif**



### Further Resources:
##### LDA
- [_Latent Dirichlet Allocation: Towards a Deeper Understanding_](http://obphio.us/pdfs/lda_tutorial.pdf), Colorado Reed, Jan 2012
- _LDA Algorithm Description_ Youtube, Scott Sullivan, Mar 2017, https://www.youtube.com/watch?v=DWJYZq_fQ2A
- _LDA Topic Models_, Youtube, Andrius Knispelis, July 2016, https://www.youtube.com/watch?v=3mHy4OSyRf0
- _Introduction to the Dirichlet Distribution and Related Processes_, B . Friqyik, A. Kapila, M. Gupta, Dec 2010, http://mayagupta.org/publications/FrigyikKapilaGuptaIntroToDirichlet.pdf

##### EM Algorithm
- _Pattern Recognition and Machine Learning_, Ch 9, Christopher M. Bishop, 2006
- _EM algorithm: how it works_, Victor Lavrenko, 2014, https://www.youtube.com/watch?v=REypj2sy_5U
- _Expectation Maximization (EM algorithm) intuitive explanation_, Youtube, pp, https://www.youtube.com/watch?v=j-3z97CYD44
